#### Computing the Linear Datamodeling Score (LDS)

In this notebook, we are going to walk through the steps of computing the Linear Datamodeling Score (LDS) for a small neural network trained on a subset of the CIFAR-10 dataset containing 5,000 samples. For more details on the LDS, please check out Section 2 in https://arxiv.org/abs/2303.14186.

In [2]:
import torch
import numpy as np
from tqdm.autonotebook import tqdm
from utils import train_on_subset, record_outputs, get_loader
np.random.seed(42)  # fix random seed for reproducibility

/tmp/ipykernel_1481195/4064116203.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


First, let's create a few subsets of the training set $\{S_1, S_2, ..., S_n\}$.
In general, we can construct these subsets in any way we desire.
Here, we are going to pick random halves of the training set. In particular, since our subset of the CIFAR-10 has 5,000 training samples, for each $S_i$ we are going to sample 2,500 i.i.d. samples from it without replacement.

In [3]:
# 1. create 20 random subsets of the training set
#    (each containing a half of the samples)
train_set_subsets = []
for i in range(20):
    subset = np.random.choice(range(5_000), 2_500, replace=False)
    train_set_subsets.append(subset)

Next, we need to obtain "ground truth" outputs for each subset. In particular, we are going to train a model on each subset $S_i$ and record its output on a few target examples of choice.

In [ ]:
# 2. train a model on each subset
#    and record its output on a target example of choice

val_loader = get_loader(split="val")
# let's use the first batch of validation data as our target examples
target_examples = next(iter(val_loader))

outputs_per_subset = []
for subset in tqdm(train_set_subsets):
    # we have abstracted away the "boring" parts of the code in utils.py
    model = train_on_subset(subset)
    # our model outputs are the margins of the model on the target examples
    # i.e., the difference between the model's logit on the correct class and
    # the log-sum-exp (think of that as a soft maximum) of logits on the incorrect classes
    outs = record_outputs(model, target_examples)
    outputs_per_subset.append(outs)

outputs_per_subset = torch.stack(outputs_per_subset)

In case you don't want to train the above models yourself, we have provided precomputed model outputs below:

In [4]:
# 2. (pre-computed version)

outputs_per_subset = torch.load('artifacts/lds_outputs_per_subset.pt')

Now, we can load in the scores from our predictive data attribution method of choice! For now, let's just use random scores as a placeholder.

In [5]:
# 3. get predicted model outputs from your attribution method for each subset;
#    here's where linearity comes into play, our prediction is the sum of
#    attribution scores across samples within the subset

# dummy scores; try to replace with your own, e.g., from the TRAK or IFs notebooks :)
dummy_attribution_scores = torch.randn(5_000, 256)  # 256 is the number of target examples

predictions_per_subset = []
for subset in train_set_subsets:
    prediction = dummy_attribution_scores[subset].sum(dim=0)
    predictions_per_subset.append(prediction)
predictions_per_subset = torch.stack(predictions_per_subset)

Finally, we compute the rank-correlation between the true model outputs and the predictions from our attribution method. In essence, we are asking how good our attribution method is at discerning which subsets $S_i$ will lead to a higher / lower 

In [6]:
# 4. evaluate the rank-correlation between the true model outputs
#    and the predictions from our attribution method
from scipy.stats import spearmanr
LDS = 0.
pval = 0.
for i in range(outputs_per_subset.shape[1]): # iterate over target examples
    LDS += spearmanr(outputs_per_subset[:, i], predictions_per_subset[:, i]).correlation

LDS = LDS / outputs_per_subset.shape[1]
print(f'LDS: {LDS:.3f}')

LDS: 0.011


Unsurprisingly, given that we loaded random scores as a placeholder, the LDS is close to 0.

In practice, we would evaluate the correlations for many target examples,
and then average the correlations across the target examples. The code below implements the many-target version of LDS efficiently.

In [ ]:
from pathlib import Path
import wget
from tqdm import tqdm

def eval_correlations(scores, tmp_path):
    masks_url = 'https://www.dropbox.com/s/x76uyen8ffkjfke/mask.npy?dl=1'
    margins_url = 'https://www.dropbox.com/s/q1dxoxw78ct7c27/val_margins.npy?dl=1'

    masks_path = Path(tmp_path).joinpath('mask.npy')
    wget.download(masks_url, out=str(masks_path), bar=None)
    # num masks, num train samples
    masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()

    margins_path = Path(tmp_path).joinpath('val_margins.npy')
    wget.download(margins_url, out=str(margins_path), bar=None)
    # num , num val samples
    margins = torch.as_tensor(np.load(margins_path, mmap_mode='r'))

    val_inds = np.arange(2_000)
    preds = masks @ scores
    rs = []
    ps = []
    for ind, j in tqdm(enumerate(val_inds)):
        r, p = spearmanr(preds[:, ind], margins[:, j])
        rs.append(r)
        ps.append(p)
    rs, ps = np.array(rs), np.array(ps)
    print(f'Correlation: {rs.mean():.3f} (avg p value {ps.mean():.6f})')
    return rs.mean()